In [1]:
from elasticsearch import Elasticsearch

es = Elasticsearch([{"host":"localhost"}, {"port":"9200"}])

print(es.ping())

True


# Create ElasticSearch Index 

In [2]:
body = {
    "mappings":{
        "properties": {
            "title_name": {"type": "keyword"},
            "title_vector": {
                "type": "dense_vector",
                "dims": 128
                }
        }
    }
}

es.indices.create(index= 'face_recognition', body=body)

RequestError: RequestError(400, 'resource_already_exists_exception', 'index [face_recognition/YVQzdoqvTN6cZP5LqBrf_Q] already exists')

# Facial database

In [3]:
import os 

files = []
for r, d, f in os.walk('tests/dataset/'):
    for file in f:
        if '.jpg' in file: 
            files.append(r+file)

# Face recognition model

In [4]:
from deepface import DeepFace
from deepface.modules import verification
from deepface.models.FacialRecognition import FacialRecognition
from deepface.commons import logger as log

# import matplotlib.pyplot as plt
import numpy as np
import cv2

2024-07-04 16:26:58.520408: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-07-04 16:26:58.520424: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
model_name = "Facenet512"
model: FacialRecognition = DeepFace.build_model(model_name=model_name)

2024-07-04 16:27:00.216950: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/ubuntu/Desktop/deepface/.venv/lib/python3.10/site-packages/cv2/../../lib64:
2024-07-04 16:27:00.216968: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2024-07-04 16:27:00.216978: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ubuntu): /proc/driver/nvidia/version does not exist
2024-07-04 16:27:00.217337: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
target_size = model.input_shape
print(f"target_size: {target_size}")

target_size: (160, 160)


In [9]:
for img_path in files:
    img = DeepFace.extract_faces(img_path=img_path)[0]["face"]
    img = cv2.resize(img, target_size)
    img = np.expand_dims(img, axis=0)
    img_representation = model.forward(img)
    print(img_representation)

AttributeError: 'list' object has no attribute 'shape'

In [8]:
model.summary()

AttributeError: 'FaceNet512dClient' object has no attribute 'summary'